In [35]:
import codecs
import glob
import pandas as pd

data_dir = "data/input/励起状態/alpha1/1A/NH/fc/*.out"
result_file = data_dir.split('励起状態')[1].split('*')[0].replace('/', '_')[1:-1]
result_dir = "data/output/" + result_file + ".csv"
paths = [p for p in glob.glob(data_dir)]
initial_key = "low   system:  model energy"
end_key = "ONIOM: calculating electric field derivatives."
df_output = pd.DataFrame()

In [36]:
for path in paths:
    # file_nameから角度の情報を取る
    file_name = path.split('/')[-1]
    torsion_angle = int(file_name.split("@")[0].split("_")[-1])

    with codecs.open(path, "r", "UTF-8", "ignore") as file:
        df = pd.read_table(file)
        df = df.rename(columns={df.columns[0]: 'Col_1'})
        # initial_keyを含む行を探す
        initial_key_df = df[df['Col_1'].str.contains(initial_key, case=False)]
        initial_key_index = initial_key_df.index[0]
        df = df.iloc[initial_key_index:]
        # initial_keyを含む行以降のend_keyを含む行を抜き出す
        end_key_df = df[df['Col_1'].str.contains(end_key)]
        # dfを更新しているのでここでの行番号は0から始まる
        end_key_index = end_key_df.index[0] - initial_key_index
        df = df.iloc[:end_key_index]
        # 該当行の抜き出しが完了
        
        # 各エネルギー値の抜き出し
        low_model = float(list(df['Col_1'])[0].split(":")[-1])
        high_model = float(list(df['Col_1'])[1].split(":")[-1])
        low_real = float(list(df['Col_1'])[2].split(":")[-1])
        high_real = float(list(df['Col_1'])[3].split("=")[-1])
        
        df_E = pd.DataFrame(data=[high_real, low_real, high_model, low_model], 
                            columns=[torsion_angle],
                            index=["high real", "low real", "high model", "low model"]).T
        df_output = pd.concat([df_output, df_E])
         

In [37]:
df_output.to_csv(result_dir)

In [38]:
df_output

,high real,low real,high model,low model
-120,-9420.653009,-9416.367700,-631.994168,-627.708859
-135,-9420.655539,-9416.378790,-631.997657,-627.720909
-150,-9420.661124,-9416.387681,-632.004266,-627.730823
-15,-9420.689397,-9416.417779,-632.023223,-627.751606
-165,-9420.667628,-9416.394673,-632.010843,-627.737888
-30,-9420.688476,-9416.411124,-632.021998,-627.744646
-45,-9420.684742,-9416.399203,-632.018236,-627.732698
-60,-9420.678232,-9416.382700,-632.012266,-627.716734
0,-9420.688005,-9416.419289,-632.022358,-627.753642
120,-9420.665758,-9416.371138,-632.005274,-627.710654
